### Imports

In [2]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
# from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "VLCS"
MODEL_NAME = "WinKawaks/vit-small-patch16-224"

Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [3]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [4]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-20 08:51:21,816 | INFO | Initialized experiment directories for VLCS
2025-10-20 08:51:21,816 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\VLCS\logs
2025-10-20 08:51:21,820 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\VLCS\checkpoints
2025-10-20 08:51:21,820 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\VLCS\plots


### Setup

In [5]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [6]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-20 08:51:23,347 | INFO | === LODO: Leaving out domain 'VOC2007' ===



=== LODO: Leaving out domain 'VOC2007' ===


Evaluating: 100%|██████████| 27/27 [00:10<00:00,  2.54it/s]
2025-10-20 08:53:27,053 | INFO | [VOC2007] Epoch 1/5 | Train - Loss: 0.5643, Cls: 0.5552, GRQO: 0.0091, Acc: 0.7832 | Val - Loss: 0.4767, Cls: 0.4748, GRQO: 0.0019, Acc: 0.8294
2025-10-20 08:53:27,156 | INFO | [VOC2007] New best val acc: 0.8294


[VOC2007] Epoch 1/5 | Train - Loss: 0.5643, Cls: 0.5552, GRQO: 0.0091, Acc: 0.7832 | Val - Loss: 0.4767, Cls: 0.4748, GRQO: 0.0019, Acc: 0.8294
[VOC2007] New best val acc: 0.8294


Evaluating: 100%|██████████| 27/27 [00:09<00:00,  2.93it/s]
2025-10-20 08:55:09,286 | INFO | [VOC2007] Epoch 2/5 | Train - Loss: 0.3343, Cls: 0.3317, GRQO: 0.0026, Acc: 0.8780 | Val - Loss: 0.5077, Cls: 0.5065, GRQO: 0.0012, Acc: 0.8095


...

[SUN09] Epoch 3/5 | Train - Loss: 0.1692, Cls: 0.1673, GRQO: 0.0019, Acc: 0.9431 | Val - Loss: 0.7304, Cls: 0.7298, GRQO: 0.0006, Acc: 0.7575


Evaluating: 100%|██████████| 26/26 [00:14<00:00,  1.78it/s]
2025-10-20 09:49:42,659 | INFO | [SUN09] Epoch 4/5 | Train - Loss: 0.1162, Cls: 0.1145, GRQO: 0.0017, Acc: 0.9607 | Val - Loss: 0.9750, Cls: 0.9745, GRQO: 0.0005, Acc: 0.7264


[SUN09] Epoch 4/5 | Train - Loss: 0.1162, Cls: 0.1145, GRQO: 0.0017, Acc: 0.9607 | Val - Loss: 0.9750, Cls: 0.9745, GRQO: 0.0005, Acc: 0.7264


Evaluating: 100%|██████████| 26/26 [00:14<00:00,  1.77it/s]
2025-10-20 09:51:26,262 | INFO | [SUN09] Epoch 5/5 | Train - Loss: 0.0785, Cls: 0.0769, GRQO: 0.0016, Acc: 0.9762 | Val - Loss: 0.9503, Cls: 0.9499, GRQO: 0.0004, Acc: 0.7745
2025-10-20 09:51:26,262 | INFO | [SUN09] Best Acc: 0.8087
2025-10-20 09:51:26,262 | INFO | ------------------------------------------------------------
2025-10-20 09:51:26,262 | INFO | LODO finished | Mean Acc: 0.8279 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\VLCS\logs\lodo_summary_20251020_095126.json


[SUN09] Epoch 5/5 | Train - Loss: 0.0785, Cls: 0.0769, GRQO: 0.0016, Acc: 0.9762 | Val - Loss: 0.9503, Cls: 0.9499, GRQO: 0.0004, Acc: 0.7745
[SUN09] Best Acc: 0.8087
------------------------------------------------------------
LODO finished | Mean Acc: 0.8279
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\VLCS\logs\lodo_summary_20251020_095126.json


### Baseline

In [7]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


2025-10-20 09:51:26,282 | INFO | Initializing ViT baseline: WinKawaks/vit-small-patch16-224


Initializing ViT baseline: WinKawaks/vit-small-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([5, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-20 09:51:27,381 | INFO | === Baseline LODO: Leaving out domain 'VOC2007' ===



=== Baseline LODO: Leaving out domain 'VOC2007' ===


2025-10-20 09:53:53,803 | INFO | [VOC2007] Epoch 1/5 | Train - Loss: 0.5182, Acc: 0.8031 | Val Acc: 0.7879


...

[SUN09] Epoch 3/5 | Train - Loss: 0.0929, Acc: 0.9691 | Val Acc: 0.7508


2025-10-20 10:26:31,706 | INFO | [SUN09] Epoch 4/5 | Train - Loss: 0.0345, Acc: 0.9905 | Val Acc: 0.7782


[SUN09] Epoch 4/5 | Train - Loss: 0.0345, Acc: 0.9905 | Val Acc: 0.7782


2025-10-20 10:28:15,553 | INFO | [SUN09] Epoch 5/5 | Train - Loss: 0.0325, Acc: 0.9899 | Val Acc: 0.7791
2025-10-20 10:28:15,553 | INFO | [SUN09] Best Val Acc: 0.7937
2025-10-20 10:28:15,553 | INFO | ------------------------------------------------------------
2025-10-20 10:28:15,553 | INFO | Baseline LODO (WinKawaks/vit-small-patch16-224) finished | Mean Acc: 0.8236


[SUN09] Epoch 5/5 | Train - Loss: 0.0325, Acc: 0.9899 | Val Acc: 0.7791
[SUN09] Best Val Acc: 0.7937
------------------------------------------------------------
Baseline LODO (WinKawaks/vit-small-patch16-224) finished | Mean Acc: 0.8236
